In [ ]:
# 安裝必要套件
# 如果已安裝可以跳過這個 cell

import sys
!{sys.executable} -m pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import math

# 初始化 MediaPipe 手部偵測工具
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

print("模組導入成功！")

✅ 模組導入成功！


In [ ]:
# 計算兩個向量之間的角度

def vector_2d_angle(v1, v2):
    """
    根據兩點的座標，計算角度
    
    參數:
        v1: 第一個向量 (x, y)
        v2: 第二個向量 (x, y)
    
    返回:
        angle: 兩向量之間的角度（度數）
    """
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    
    try:
        # 使用餘弦定理計算角度
        angle_ = math.degrees(
            math.acos(
                (v1_x*v2_x + v1_y*v2_y) / 
                (((v1_x**2 + v1_y**2)**0.5) * ((v2_x**2 + v2_y**2)**0.5))
            )
        )
    except:
        angle_ = 180
    
    return angle_

print("vector_2d_angle 函數定義完成！")


✅ vector_2d_angle 函數定義完成！


In [ ]:
# 計算手指角度

def hand_angle(hand_):
    """
    根據傳入的 21 個手部節點座標，計算五根手指的角度
    
    參數:
        hand_: 包含 21 個手部關鍵點的座標列表
    
    返回:
        angle_list: 五根手指的角度列表 [大拇指, 食指, 中指, 無名指, 小拇指]
    """
    angle_list = []
    
    # 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[2][0])), (int(hand_[0][1]) - int(hand_[2][1]))),
        ((int(hand_[3][0]) - int(hand_[4][0])), (int(hand_[3][1]) - int(hand_[4][1])))
    )
    angle_list.append(angle_)
    
    # 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[6][0])), (int(hand_[0][1]) - int(hand_[6][1]))),
        ((int(hand_[7][0]) - int(hand_[8][0])), (int(hand_[7][1]) - int(hand_[8][1])))
    )
    angle_list.append(angle_)
    
    # 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[10][0])), (int(hand_[0][1]) - int(hand_[10][1]))),
        ((int(hand_[11][0]) - int(hand_[12][0])), (int(hand_[11][1]) - int(hand_[12][1])))
    )
    angle_list.append(angle_)
    
    # 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[14][0])), (int(hand_[0][1]) - int(hand_[14][1]))),
        ((int(hand_[15][0]) - int(hand_[16][0])), (int(hand_[15][1]) - int(hand_[16][1])))
    )
    angle_list.append(angle_)
    
    # 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[18][0])), (int(hand_[0][1]) - int(hand_[18][1]))),
        ((int(hand_[19][0]) - int(hand_[20][0])), (int(hand_[19][1]) - int(hand_[20][1])))
    )
    angle_list.append(angle_)
    
    return angle_list

print("hand_angle 函數定義完成！")


✅ hand_angle 函數定義完成！


In [ ]:
# 手勢識別

def hand_pos(finger_angle):
    """
    根據手指角度的列表內容，返回對應的手勢名稱
    
    參數:
        finger_angle: 五根手指的角度列表
    
    返回:
        str: 手勢名稱
    
    角度規則: 小於 50 表示手指伸直，大於等於 50 表示手指捲縮
    """
    f1 = finger_angle[0]   # 大拇指角度
    f2 = finger_angle[1]   # 食指角度
    f3 = finger_angle[2]   # 中指角度
    f4 = finger_angle[3]   # 無名指角度
    f5 = finger_angle[4]   # 小拇指角度

    # 手勢判斷邏輯
    if f1<50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return 'good'        # 讚
    elif f1>=50 and f2>=50 and f3<50 and f4>=50 and f5>=50:
        return 'no!!!'       # 比中指（會加馬賽克）
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5<50:
        return 'ROCK!'       # 搖滾
    elif f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return '0'           # 拳頭
    elif f1>=50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '1'           # 1
    elif f1>=50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '2'           # 2
    elif f1>=50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'          # OK
    elif f1<50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'          # OK (另一種)
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5>50:
        return '3'           # 3
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '4'           # 4
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '5'           # 5
    elif f1<50 and f2>=50 and f3>=50 and f4>=50 and f5<50:
        return '6'           # 6
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '7'           # 7
    elif f1<50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '8'           # 8
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5>=50:
        return '9'           # 9
    else:
        return ''            # 無法識別

print("hand_pos 函數定義完成！")
print("支援的手勢: good, ROCK!, 0-9, ok, pink, no!!!")  


✅ hand_pos 函數定義完成！
📋 支援的手勢: good, ROCK!, 0-9, ok, pink, no!!!


In [ ]:
# 啟動手勢識別

# 初始化攝影機
cap = cv2.VideoCapture(0)
fontFace = cv2.FONT_HERSHEY_SIMPLEX  # 文字字型
lineType = cv2.LINE_AA               # 文字邊框

print("正在啟動攝影機...")
print("按 'q' 鍵退出程式")

# 啟用 MediaPipe 手部偵測
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("無法開啟攝影機")
        exit()
    
    w, h = 540, 310  # 影像尺寸
    
    while True:
        ret, img = cap.read()
        img = cv2.resize(img, (w, h))  # 縮小尺寸，加快處理效率
        
        if not ret:
            print("❌ 無法讀取影像")
            break
        
        # 轉換成 RGB 色彩供 MediaPipe 處理
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img2)
        
        # 如果偵測到手部
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                finger_points = []  # 記錄手指節點座標
                fx = []             # 記錄所有 x 座標
                fy = []             # 記錄所有 y 座標
                
                # 取得 21 個手部關鍵點
                for i in hand_landmarks.landmark:
                    x = i.x * w
                    y = i.y * h
                    finger_points.append((x, y))
                    fx.append(int(x))
                    fy.append(int(y))
                
                if finger_points:
                    # 計算手指角度
                    finger_angle = hand_angle(finger_points)
                    
                    # 識別手勢
                    text = hand_pos(finger_angle)
                    
                    # 特殊處理：如果是比中指，加馬賽克
                    if text == 'no!!!':
                        x_max = max(fx)
                        y_max = max(fy)
                        x_min = min(fx) - 10
                        y_min = min(fy) - 10
                        
                        # 邊界檢查
                        if x_max > w: x_max = w
                        if y_max > h: y_max = h
                        if x_min < 0: x_min = 0
                        if y_min < 0: y_min = 0
                        
                        # 製作馬賽克效果
                        mosaic_w = x_max - x_min
                        mosaic_h = y_max - y_min
                        mosaic = img[y_min:y_max, x_min:x_max]
                        mosaic = cv2.resize(mosaic, (8, 8), interpolation=cv2.INTER_LINEAR)
                        mosaic = cv2.resize(mosaic, (mosaic_w, mosaic_h), interpolation=cv2.INTER_NEAREST)
                        img[y_min:y_max, x_min:x_max] = mosaic
                    else:
                        # 顯示手勢文字
                        cv2.putText(img, text, (30, 120), fontFace, 5, (255, 255, 255), 10, lineType)

        # 顯示影像
        cv2.imshow('Hand Gesture Recognition', img)
        
        # 按 'q' 退出
        if cv2.waitKey(5) == ord('q'):
            print("\n程式結束")
            break

cap.release()
cv2.destroyAllWindows()
print("攝影機已關閉")

🎥 正在啟動攝影機...
⌨️  按 'q' 鍵退出程式

👋 程式結束
✅ 攝影機已關閉
